In [2]:
from scapy.all import *

## Scapy Examples

<details>
<summary>Table of Contents</summary>

- [UDP 예시](#UDP-예시)
- [TCP 예시](#TCP-예시)
- [TCP Control Bits](#TCP-Control-Bits)
- [IP 예시](#IP-예시)
- [ICMP 예시](#ICMP-예시)
- [여러 Protocol의 Header 결합](#여러-Protocol의-Header-결합)
- [IP 예시](#IP-예시)
- [ICMP 예시](#ICMP-예시)
- [여러 Protocol의 Header 결합](#여러-Protocol의-Header-결합)
- [ARP 예시](#ARP-예시)
- [Ether 예시](#Ether-예시)


</details>

---

### UDP 예시
- `UDP()` 함수를 통해 UDP Protocol의 Datagram을 생성할 수 있다.
- 생성된 Datagram은 객체의 `show()` 함수를 이용하면 정보를 출력한다.

In [3]:
# UDP Header 생성
datagram = UDP()

# UDP Header 생성 결과 출력
datagram.show()

###[ UDP ]### 
  sport     = domain
  dport     = domain
  len       = None
  chksum    = None



- Datagram의 정보는 Python 내장 함수인 `del()` 함수를 통해 해당 요소의 초기화가 가능하다.
- sport는 Source Port, dport는 Destination Port의 줄임말이다.

In [4]:
# sport, dport 설정
datagram.sport = 12345
datagram.dport = 54321
# 설정된 속성 확인
datagram.show()

# sport, dport 삭제
del (datagram.sport, datagram.dport)
# 삭제 이후 정보 확인
datagram.show()

###[ UDP ]### 
  sport     = 12345
  dport     = 54321
  len       = None
  chksum    = None

###[ UDP ]### 
  sport     = domain
  dport     = domain
  len       = None
  chksum    = None



### TCP 예시
- `TCP()` 함수를 통해 TCP Protocol의 Segment를 생성할 수 있다.
- TCP Protocol은 TCP의 특성상 UDP보다 더 많은 정보를 포함하고 있다.
- `TCP()` 생성자를 호출할 때, 각종 Parameter를 지정하면 Packet의 구조를 변경할 수 있다. (다른 Protocol도 동일)

In [5]:
# TCP Header 생성
segment = TCP()

# TCP의 정보는 UDP보다 더 많음
segment.show()

###[ TCP ]### 
  sport     = ftp_data
  dport     = http
  seq       = 0
  ack       = 0
  dataofs   = None
  reserved  = 0
  flags     = S
  window    = 8192
  chksum    = None
  urgptr    = 0
  options   = []



- 아래는 `TCP()` 생성자의 Parameter에 값을 할당해서 세부 정보를 변경한 예시이다.

In [6]:
# Parameter를 전달해서 각종 정보를 설정할 수 있음
# Parameter에 설정되는 것은 TCP Packet 구조의 내용을 변형하는 것
segment = TCP(sport=12345, dport=22, dataofs=5, window=2918)

segment.show()

###[ TCP ]### 
  sport     = 12345
  dport     = ssh
  seq       = 0
  ack       = 0
  dataofs   = 5
  reserved  = 0
  flags     = S
  window    = 2918
  chksum    = None
  urgptr    = 0
  options   = []



### TCP Control Bits

- TCP Protocol의 Header에는 아래와 같은 총 6 bit의 Control Bits가 포함되어 있다.

| 플래그 | 값 | 설명                |
|--------|----|---------------------|
| URG    | 32 | 긴급 데이터를 나타냄  |
| ACK    | 16 | 확인 응답을 나타냄    |
| PSH    | 8  | 버퍼를 비우라는 신호 |
| RST    | 4  | 연결 재설정을 나타냄 |
| SYN    | 2  | 연결 설정을 나타냄   |
| FIN    | 1  | 연결 종료를 나타냄   |

- 각 Flag의 상태는 앞에서부터 채워진다.
- 위의 표에서 해당 Flag를 활성화 한다면 1, 그렇지 않다면 0을 설정하여 연속된 숫자를 만들 수 있다.
- 아래는 모든 비트가 0인 예시이다.

| Bit | 5   | 4   | 3   | 2   | 1   | 0   |
|-----|-----|-----|-----|-----|-----|-----|
| Value | 0   | 0   | 0   | 0   | 0   | 0   |
| Flag  | URG | ACK | PSH | RST | SYN | FIN |

- 만약 `ACK`와 `SYN` bit가 설정된 Control Bits를 만들고 싶다면, 아래와 같은 구성이 된다.
- 이는 10진수로 18이며, 16진수로는 0x12로 계산할 수 있다.

| Bit | 5   | 4   | 3   | 2   | 1   | 0   |
|-----|-----|-----|-----|-----|-----|-----|
| Value | 0   | 1   | 0   | 0   | 1   | 0   |
| Flag  | URG | ACK | PSH | RST | SYN | FIN |

In [7]:
# TCP 패킷의 Control Bits는 각 상태를 나타내는 6개의 비트를
# 순서대로 변환하여 flags의 Parameter로 전달할 수 있다.
segment = TCP(flags=0x12)

segment.show()

###[ TCP ]### 
  sport     = ftp_data
  dport     = http
  seq       = 0
  ack       = 0
  dataofs   = None
  reserved  = 0
  flags     = SA
  window    = 8192
  chksum    = None
  urgptr    = 0
  options   = []



- Python에서는 가장 작은 단위가 1 byte이다.
- Offset에 5라는 값을 넣고싶은 경우, Offset을 Shift 연산하여 계산할 수 있다.
- Reserved는 IP Header에서 3 bit의 크기 값으로 미래 확장을 위해 예약된 Field이다.
- Reserved가 현재는 사용되지 않고 있어서, 0으로 설정된다.

In [8]:
# Offset 값 설정
offset = 5
reserved = 0

offset_reserved = offset << 4 + reserved
offset_reserved

80

- 아래는 Shift 연산을 통해 Control Bits를 만드는 예시이다.
- 각 자리를 맞는 Bit에 Shift 연산하여 Flags 값을 계산할 수 있다.

| Bit   | 5   | 4   | 3   | 2   | 1   | 0   |
| ----- | --- | --- | --- | --- | --- | --- |
| Value | 1   | 1   | 1   | 1   | 1   | 1   |
| Flag  | URG | ACK | PSH | RST | SYN | FIN |


In [9]:
urg = ack = psh = rst = syn = fin = flags = 1

# Shift 연산을 통해 비트만큼 Shift하여 각 자리를 지정해줄 수 있다.
flags = (urg << 5) + (ack << 4) + (psh << 3) + (rst << 2) + (syn << 1) + fin

# Flags는 Binary 값이다.
bin(flags)

'0b111111'

### IP 예시
- `IP()` 함수를 통해 IP의 Packet을 생성할 수 있다.
- `len` : IP header와 TCP Header와 TCP Payload 등의 길이가 나타난다.
- `frag (fragmentation)` : 해당 Header가 MTU로 쪼개진 Header인지에 대한 여부이다.
- `proto (protocol)` : Data에 담겨있는 Packet이 어떤 Protocol을 사용하는지에 대한 정보이다.
- `ihl (IP Header Length)` : IP Header의 길이를 나타낸다. 32 bit word 단위로 표현되며, 그 개수를 나타낸다.
- `id` : `frag`의 Packet 식별자를 나타낸다. 분할된 Packet의 경우, 원래 Packet을 재조립하기 위해 사용한다.
- `proto` : IP Packet에 포함된 Data의 Protocol을 나타낸다. 값이 6이면, TCP Protocol을 사용한다.

In [10]:
# IP Header 생성
packet = IP()

packet.show()

###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = hopopt
  chksum    = None
  src       = 127.0.0.1
  dst       = 127.0.0.1
  \options   \



- IP Packet도 마찬가지로, 아래의 Parameter를 지정하여 나타낼 수 있다.

In [11]:
packet = IP(ihl=20, id=2222, proto=6)

packet.show()

###[ IP ]### 
  version   = 4
  ihl       = 20
  tos       = 0x0
  len       = None
  id        = 2222
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = tcp
  chksum    = None
  src       = 127.0.0.1
  dst       = 127.0.0.1
  \options   \



### ICMP 예시

- ICMP는 Network 상에서 오류를 보고하거나, Echo 요청 및 응답(Ping)에 사용되거나, 경로 MTU 발견 등으로 사용되는 Protocol이다.
- 즉, 각종 정보를 전달하기 위한 목적을 가진다.
- `ICMP()` 생성자를 통해 ICMP Packet을 생성할 수 있다.
- `show()` 함수를 통해 나타난 결과에서 `type`이 `echo-request`이라면 연결이 잘 되어 있는 경우이다.

In [12]:
# ICMP Packet 생성
packet = ICMP()

packet.show()

###[ ICMP ]### 
  type      = echo-request
  code      = 0
  chksum    = None
  id        = 0x0
  seq       = 0x0



### 여러 Protocol의 Header 결합
- 연산자 `/`를 이용하여 여러 Protocol의 생성자를 호출하면, 해당 Protocol을 가지는 하나의 Packet을 생성할 수 있다.
- `Raw()` 객체의 생성자를 이용하여 `load` Parameter를 전달하면, 메시지를 전달할 수 있다.

In [13]:
DESTINATION_IP = '10.0.0.1'
RAW_MESSAGE = 'test'

packet = IP(dst=DESTINATION_IP) / ICMP() / Raw(load=RAW_MESSAGE)

packet.show()

###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = icmp
  chksum    = None
  src       = xxx.xxx.xxx.xxx
  dst       = 10.0.0.1
  \options   \
###[ ICMP ]### 
     type      = echo-request
     code      = 0
     chksum    = None
     id        = 0x0
     seq       = 0x0
###[ Raw ]### 
        load      = 'test'



- 위에서 생성된 Packet은 `sr1()` 메서드를 사용하여 실제로 전달할 수 있다.

In [ ]:
sr1(packet)

# 마찬가지로, UDP를 이용할 수도 있다.
packet = IP() / UDP() / Raw(load=RAW_MESSAGE)
sr1(packet)

### ARP 예시
- ARP Protocol은 IP 주소와 MAC 주소간의 Mapping을 수행하는 Protocol이다.
- `ARP()` 생성자를 호출하여 ARP Packet을 만들 수 있다.

In [16]:
# ARP Packet 생성
packet = ARP()

packet.show()

###[ ARP ]### 
  hwtype    = 0x1
  ptype     = IPv4
  hwlen     = None
  plen      = None
  op        = who-has
  hwsrc     = xx:xx:xx:xx:xx:xx
  psrc      = xxx.xxx.xxx.xxx
  hwdst     = 00:00:00:00:00:00
  pdst      = 0.0.0.0



- OSI Layer 중, L2 계층에서는 MAC 주소로 Data를 주고 받는다.
- `ARP()` 객체를 이용하여 Packet을 만들면 이를 변형할 수 있다.
- 만약 알고 싶은 MAC 주소가 있고, 해당 IP 주소를 가지고 있다면, 객체를 생성하여 네트워크에 전송한 후 MAC 주소를 알 수 있다.

In [ ]:
packet = ARP(pdst=DESTINATION_IP)

sr1(packet)

### Ether 예시
- L2 계층에서는 주로 Ethernet Protocol을 사용한다.
- `Ether()` 생성자를 통해 Frame을 생성할 수 있다.

In [20]:
# Ether Frame 생성
frame = Ether()

frame.show()

###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = xx:xx:xx:xx:xx:xx
  type      = 0x9000



- 아래는 전체를 만든 예시이다.
- 이러한 경우, `show()` 함수는 전체 계층을 확인할 수 있게 된다.

In [21]:
frame = Ether() / IP() / UDP() / Raw(load=RAW_MESSAGE)

frame.show()

###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 00:00:00:00:00:00
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = None
     tos       = 0x0
     len       = None
     id        = 1
     flags     = 
     frag      = 0
     ttl       = 64
     proto     = udp
     chksum    = None
     src       = 127.0.0.1
     dst       = 127.0.0.1
     \options   \
###[ UDP ]### 
        sport     = domain
        dport     = domain
        len       = None
        chksum    = None
###[ Raw ]### 
           load      = 'test'



- 아래의 예제는 `Ether()` 의 MAC 주소를 Broadcast 하는 코드이다.
- 하지만, 받은 Packet은 많지만 적합한 MAC 주소가 없기 때문에 응답이 오지 않고 Network의 Node는 이를 무시한다.

In [ ]:
frame = Ether() / ARP()

sr1(frame)